In [2]:
!cp -r /content/drive/MyDrive/1DATASETS_new/Fake_Disaster_Dataset /content/

## AIGCDetectBenchmark


## Install requirement

In [3]:
!git clone https://github.com/vohoaidanh/ADOF.git

Cloning into 'ADOF'...
remote: Enumerating objects: 873, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 873 (delta 45), reused 48 (delta 41), pack-reused 817 (from 1)
Receiving objects: 100% (873/873), 28.78 MiB | 39.30 MiB/s, done.
Resolving deltas: 100% (574/574), done.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd ADOF

/content/ADOF


In [ ]:
!pip install -r requirements.txt
!pip install tensorboardX -q


In [6]:
# Without google colab
COLAB = True
if not COLAB:
  !pip install gdown -q
  !apt-get install -y unzip -q
  !apt-get install -y zip -q
  !pip install tensorboardX -q
  !pip install regex -q
  !pip install imageio -q
  !pip install opencv-python -q
  !apt-get install -y libgl1-mesa-glx -q
  !pip install scikit-learn -q
  !pip install scikit-image -q
  !pip install ftfy -q
  !pip install natsort -q
  !pip install blobfile -q

## Download Data

In [ ]:
#Download Train, val set
# https://drive.google.com/file/d/1JW-4upuC3Tqq9awz8gnRWEf61NncC8JT/view?usp=drive_link    (Full train, val set)
# https://drive.google.com/file/d/1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z/view?usp=drive_link     (Small tran, val set)
import gdown
DATASET_PATH = 'datasets'
TRAIN_SET_PATH = 'ForenSynths_train'
file_id = '1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z' #Progan train/val 4 class [car, cat, chair, horse]
filename_train = 'progan_train.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, filename_train, quiet=False)
#Unzip
!mkdir -p $DATASET_PATH/$TRAIN_SET_PATH
!mkdir -p $DATASET_PATH/$TRAIN_SET_PATH/test
!unzip -q $filename_train -d $DATASET_PATH/$TRAIN_SET_PATH

Downloading...
From (original): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z
From (redirected): https://drive.google.com/uc?id=1JLdFyM7JnaUa8y4wBOwNGs3rn4qKH58Z&confirm=t&uuid=6c860d9b-9fab-4829-b947-97540cd5c9ef
To: /content/ADOF/progan_train.zip
100%|██████████| 2.64G/2.64G [00:26<00:00, 98.2MB/s]


In [ ]:
#Download small Test set (this is a small testset just use for reference during training)

import gdown
file_id = '1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4' #test_set
filename_test = 'test_set.zip'  # Desired file name and extension
# Construct the download URL
url = f'https://drive.google.com/uc?id={file_id}'
# Download the file
gdown.download(url, filename_test, quiet=False)
#Unzip
!unzip -q $filename_test -d $DATASET_PATH/$TRAIN_SET_PATH/test

Downloading...
From (original): https://drive.google.com/uc?id=1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4
From (redirected): https://drive.google.com/uc?id=1-02FPvdTaQFDMatEvXAeKLCmnwW6RLx4&confirm=t&uuid=8c039f27-2a43-45b3-b7e4-a856f0c5f5d6
To: /content/ADOF/test_set.zip
100%|██████████| 1.68G/1.68G [00:20<00:00, 83.5MB/s]


UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [ ]:
!unzip -q $filename_test -d $DATASET_PATH/$TRAIN_SET_PATH/test

## Train

In [ ]:
#experiment-01-no-filter
!find $DATASET_PATH -type d -name "*ipynb*" -exec rm -r {} +
!python train.py \
--name adof-progan-4class- \
--dataroot $DATASET_PATH/$TRAIN_SET_PATH \
--num_thread 2 \
--classes car,cat,chair,horse --batch_size 32 --delr_freq 5 --lr 0.0002 --niter 30

----------------- Options ---------------
                     arch: res50                         
               batch_size: 32                            	[default: 64]
                    beta1: 0.9                           
                blur_prob: 0                             
                 blur_sig: 0.5                           
          checkpoints_dir: ./checkpoints                 
                class_bal: False                         
                  classes: car,cat,chair,horse           	[default: ]
           continue_train: False                         
                 cropSize: 224                           
                 data_aug: False                         
                 dataroot: datasets/ForenSynths_train    	[default: ./dataset/]
                delr_freq: 5                             	[default: 20]
          earlystop_epoch: 15                            
                    epoch: latest                        
              epoch_count:

## Evaluation

In [ ]:
!chmod +x download_dataset.sh
!sed -i 's/\r$//' download_dataset.sh
!./download_dataset.sh

In [16]:
# You should update the test set directory in the "test.py" file.
!find /content/Fake_Disaster_Dataset -type d -name "*ipynb*" -exec rm -r {} +

!python inference.py --test_root /content/train-free/test/stable_diffusion_xl --model_path ./weights/ADOF_model_epoch_9.pth  \
  --output_csv results_train_free_stable_diffusion_xl.csv \
  --batch_size 64 \
  --threshold 0.5


Streaming output truncated to the last 5000 lines.
/content/train-free/test/stable_diffusion_xl/earthquake/997.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/966.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/774.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/1412.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/233.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/1212.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/1404.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/915.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/904.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/499.jpg -> fake (prob=1.0000)
/content/train-free/test/stable_diffusion_xl/earthquake/1259.png -> fake (prob=1.0000)

In [10]:
!unzip -q /content/Fake_Disaster_Dataset/train-free.zip -d /content/